In [1]:
import numpy as np
import numpy.typing as npt
# from numba import jit, njit, prange
import pickle
from dataclasses import dataclass, asdict, field
import argparse
import hashlib
import json
from os import listdir
from os.path import isfile, join
# import pandas as pd
from pathlib import Path
from typing import Any
import time
import sys
import copy
import concurrent.futures as cf

from src.dataclass import (
    Input, Lattice, Parameter, Train, Save,
    Processed_Input, Topology, Conjugate, Result, summarize_results
)
from src.manage_data import save_result, save_log, load_result
from src.initial_state import get_initial_state
from src.process_input import (get_processed_input, get_T_and_H, get_J)
from src.metropolis import execute_metropolis_update
from src.function import time_correlation
from src.process_output import get_result

In [2]:
def run_ensemble(
    input: Input,
    processed_input: Processed_Input,
    # J: npt.NDArray,
    ensemble_num: int
) -> tuple[int, Result]:
    # tuple[int, tuple, npt.NDArray[np.float64], int]

    (size, dimension, iteration, sweep,
     measurement, interval, recent, threshold, max_workers) = (
        input.lattice.size,
        input.lattice.dimension,
        input.train.iteration,
        input.train.sweep,
        input.train.measurement,
        input.train.interval,
        input.train.recent,
        input.train.threshold,
        input.train.max_workers,
    )

    begin = time.perf_counter()

    J = get_J(input, processed_input)  # get coupling parameter

    initial = get_initial_state(input)  # initialize state
    update = initial.copy()
    autocorr, autocorr_len = np.empty(
        iteration+1, dtype=np.float64), 1  # autocorrelation
    autocorr[0] = time_correlation(initial, initial, size**dimension)

    now = time.perf_counter()
    # Removing initial state effect until autocorrelation satsisfies certain criteria
    for _ in range(iteration):
        update = execute_metropolis_update(
            input, processed_input, J, update)
        autocorr[autocorr_len] = np.abs(time_correlation(
            update, initial, size**dimension))
        autocorr_len += 1
        if autocorr_len > recent:
            temp = autocorr[autocorr_len-recent:autocorr_len]
            if np.average(temp) < threshold or np.std(temp) < threshold:
                break

    if ensemble_num % max_workers == 1:
        print(
            f"initial effect removed, iter: {autocorr_len}, time: {time.perf_counter()-now}s, avg: {np.average(autocorr[autocorr_len-recent:autocorr_len])}, std: {np.std(autocorr[autocorr_len-recent:autocorr_len])}")

    now = time.perf_counter()
    # Collect raw output after performing metropolis update
    raw_output = np.empty((measurement, size**dimension), dtype=np.complex128)
    for i in range(sweep):
        update = execute_metropolis_update(input, processed_input, J, update)
        if autocorr_len <= iteration:
            autocorr[autocorr_len] = np.abs(
                time_correlation(update, initial, size**dimension))
            autocorr_len += 1
        if i % interval == interval - 1:
            # ! .copy() maybe not essential?
            raw_output[int(i/interval)] = update.copy()

    if ensemble_num % max_workers == 1:
        print(
            f"raw output collected, iter: {sweep}, time: {time.perf_counter()-now}s")

    now = time.perf_counter()
    # process raw output
    result = get_result(input, processed_input, raw_output, J)

    if ensemble_num % max_workers == 1:
        print(f"raw output processed, time: {time.perf_counter()-now}s")
    result.autocorrelation = np.array(autocorr)
    result.time = time.perf_counter() - begin

    return ensemble_num, result

In [3]:
def sampling(
    input: Input,
    processed_input: Processed_Input,
) -> None:

    max_workers, ensemble = (
        input.train.max_workers,
        input.train.ensemble,
    )

    with cf.ProcessPoolExecutor(max_workers=max_workers) as executor:
        futures = [
            executor.submit(run_ensemble, input, processed_input, i + 1)
            for i in range(ensemble)  # execute each ensemble
        ]

        ensemble_results: list[Result] = []
        finished = 0

        for future in cf.as_completed(futures):
            finished += 1
            ensemble_num, single_result = future.result()
            ensemble_results.append(single_result)

            if ensemble_num % max_workers == 1:
                print(f"{ensemble_num}: {single_result}")

    result = summarize_results(ensemble_results)

    save_log(input, result)

    if input.save.save:
        save_result(input, result)

    print(input)
    print(result, "\n")

In [4]:
def experiment(args: argparse.Namespace) -> None:
    lattice = Lattice(
        args.state, args.size, args.dimension,
        args.ghost, args.initial)
    parameter = Parameter(
        args.Tc, args.Hc, args.Jm, args.Jv, args.mode,
        args.variable, args.multiply, args.base, args.exponent)
    train = Train(
        args.iteration, args.sweep, args.measurement, args.interval,
        args.ensemble, args.max_workers, args.threshold, args.recent)
    save = Save(args.environment, args.location, args.save)

    input = Input(lattice, parameter, train, save)
    input.parameter.T, input.parameter.H = args.T, args.H

    now = time.perf_counter()
    processed_input = get_processed_input(input)  # process input
    print(f"input processed, time: {time.perf_counter()-now}s")

    input.parameter.T, input.parameter.H = get_T_and_H(
        input)  # get temperature and external field

    print(input, "\n")
    sampling(input, processed_input)

In [5]:
parser = argparse.ArgumentParser()
args = parser.parse_args("")

"""
Lattice Condition
"""
args.state = 3
args.size = 128
args.dimension = 2
args.ghost = 0
args.initial = "uniform" # "uniform", "random"

"""
Parameter Condition
"""
args.T, args.H = 1.0, 0.0
# q=2: 2.2692, q=3: 1.4925
# args.Tc = 1/((1-1/args.state)*np.log(1+np.sqrt(args.state)))
args.Tc = 2.2
args.Hc = 0.0
args.Jm = 1.0
args.Jv = 0.0

args.mode = "normal"  # 'normal', 'critical' and 'manual'
args.variable = "T"  # 'T', 'H'
args.multiply = 3*0.1**4
args.base = 2.0
args.exponent = 4

"""
Train Condition
"""
args.iteration = 2**10
args.measurement = 2**14 # total number of measurement
args.interval = 2**0 # measure once in every "interval" iteration
args.sweep = args.measurement * args.interval # total number of iteration
args.max_workers = 1
args.ensemble = args.max_workers * 2**0
args.threshold = 0.1**2
args.recent = 10**2

"""
Save Condition
"""
args.environment = "local" # "server" or "local"
args.location = "result" # "result" of "temp"
args.save = True  # True or False

experiment(args)

name1, list1 = 'exponent', [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12]
# name1, list1 = 'T', [2.1, 2.2, 2.3, 2.4]

# for var1 in list1:
#     setattr(args, name1, var1)
#     experiment(args)

input processed, time: 1.8395763860025909s
Input(lattice=Lattice(state=4, size=12, dimension=3, ghost=0, initial='uniform'), parameter=Parameter(T=2.2012, H=0.0, Tc=2.2, Hc=0.0, Jm=1.0, Jv=0.0, mode='normal', variable='T', multiply=0.00030000000000000003, base=2.0, exponent=2), train=Train(iteration=1024, sweep=16384, measurement=16384, interval=1, ensemble=256, max_workers=8, threshold=0.010000000000000002, recent=100), save=Save(environment='local', location='result', save=True)) 

initial effect removed, iter: 1025, time: 0.8788739159936085s, avg: 0.5169560185185185, std: 0.059068244797944904
order parameter processed, 0.050869269005488604s
order parameter processed, 0.07442866299243178s
raw output collected, iter: 16384, time: 5.528733496001223s
order parameter processed, 0.10120446399378125s
order parameter processed, 0.14685010300308932s
order parameter processed, 0.16255981801077724s
order parameter processed, 0.15783602399460506sorder parameter processed, 0.16378943400923163s

